In [9]:
# GESTION DEL TRANSPORTE INTERMODAL: MODELO SINCROMODAL SIMPLIFICADO
# AUTOR: ASHTON IAN HETHERINGTON
# MASTER UNIVERSITARIO EN INGENIERIA INDUSTRIAL - UPCT
# TRABAJO FIN DE MASTER
!pip install cvxpy
!pip install cvxopt
!pip install mss
from itertools import product
import numpy as np
import cvxpy as cp
import cvxopt
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import networkx as nx
import webbrowser
#from mss.windows import MSS as mss
from mss import mss


class MSS:
     
    def __init__(self, framework='CVXPY'):

        # Definicion de los parametros

        self.auxTerminal = None
        self.auxTiempo = None
        self.mercancia = None
        self.indiceTerminal = None
        self.terminalIndice = None
        self.fechaMax = None
        self.fechaMin = None
        self.cTransporte = None
        self.cFijosTransp = None
        self.tTransporte = None
        self.vContenedor = None
        self.cAlmacen = None
        self.vPedido = None
        self.cPedido = None
        self.fechaLimEntrega = None
        self.terminalOrigen = None
        self.terminalDestino = None
        self.tInicio = None
        self.aranceles = None
        self.cTransito = None
        self.numRuta = None
        self.rutasDisp = None

        # Variables de decision

        self.var_1 = None
        self.x = None
        self.var_2 = None
        self.y = None
        self.var_3 = None
        self.z = None        
        # Solucion y resultados

        self.xs = None
        self.ys = None
        self.zs = None
        self.cFinalAlmacen = None
        self.costeTransporte = None
        self.cImpuestos = None
        self.solucion_ = None
        self.horaLlegada_ = None
        self.valorObjFun = None

        # Variables de ayuda

        self.var_1_ubicacion = None
        self.var_2_ubicacion = None
        self.var_3_ubicacion = None

        self.framework = framework
        
    def definir_param(self, ruta, pedido):
        '''DEFINE LOS PARAMETROS DEL MODELO BASADO EN LA INFORMACION DE LA HOJA EXCEL QUE CONTIENE LAS RUTAS Y PEDIDOS'''
        bigM = 100000
        ruta = ruta[ruta['Feasibility'] == 1]
        ruta['Costes de Almacenamiento'][ruta['Costes de Almacenamiento'].isnull()] = bigM
        ruta = ruta.reset_index()

        sTerminales = set(ruta['Terminal Origen']) | set(ruta['Terminal Destino'])

        self.auxTerminal = len(sTerminales)
        self.terminalIndice = dict(zip(range(len(sTerminales)), sTerminales))
        self.indiceTerminal = dict(
            zip(self.terminalIndice.values(), self.terminalIndice.keys()))

        self.fechaMax = np.max(pedido['Fecha Entrega'])
        self.fechaMin = np.min(pedido['Fecha Pedido'])
        self.auxTiempo = (self.fechaMax - self.fechaMin).days
        primerDia = self.fechaMin.weekday() + 1
        weekday = np.mod((np.arange(self.auxTiempo) + primerDia), 7)
        weekday[weekday == 0] = 7
        listaFechasDias = {i: [] for i in range(1, 8)}
        for i in range(len(weekday)):
            listaFechasDias[weekday[i]].append(i)
        for i in listaFechasDias:
            listaFechasDias[i] = json.dumps(listaFechasDias[i])

        origen = list(ruta['Terminal Origen'].replace(self.indiceTerminal))
        destino = list(ruta['Terminal Destino'].replace(self.indiceTerminal))
        listaFecha = list(ruta['Weekday'].replace(
            listaFechasDias).apply(json.loads))

        self.mercancia = pedido.shape[0]
        self.cTransporte = np.ones(
            [self.auxTerminal, self.auxTerminal, self.auxTiempo]) * bigM
        self.cFijosTransp = np.ones(
            [self.auxTerminal, self.auxTerminal, self.auxTiempo]) * bigM
        self.tTransporte = np.ones(
            [self.auxTerminal, self.auxTerminal, self.auxTiempo]) * bigM

        for i in range(ruta.shape[0]):
            self.cTransporte[origen[i], destino[i],
                listaFecha[i]] = ruta['Cost'][i]
            self.cFijosTransp[origen[i], destino[i],
                listaFecha[i]] = ruta['Costes Fijos Transporte'][i]
            self.tTransporte[origen[i], destino[i],
                listaFecha[i]] = ruta['Time'][i]

        self.cTransito = np.ones([self.auxTerminal, self.auxTerminal]) * bigM
        self.cTransito[origen, destino] = ruta['Tasas Internacionales']
                # Reduccion del numero de contenedores de las rutas inviables a un numero similar a bigM

        self.vContenedor = np.ones([self.auxTerminal, self.auxTerminal]) * 0.1
        self.vContenedor[origen, destino] = ruta['Tamaño Contenedor']
        self.vContenedor = self.vContenedor.reshape(self.auxTerminal, self.auxTerminal, 1)
        self.cAlmacen = ruta[['Terminal Origen', 'Costes de Almacenamiento']].drop_duplicates()
        self.cAlmacen['index'] = self.cAlmacen['Terminal Origen'].replace(self.indiceTerminal)
        self.cAlmacen = np.array(self.cAlmacen.sort_values(
            by='index')['Costes de Almacenamiento'])
        self.vPedido = np.array(pedido['Volumen Pedido'])
        self.cPedido = np.array(pedido['Valor Mercancia'])
        self.fechaLimEntrega = np.array(
            (pedido['Fecha Entrega'] - self.fechaMin).dt.days)
        self.terminalOrigen = np.array(pedido['Remitente'].replace(self.indiceTerminal))
        self.terminalDestino = np.array(pedido['Destinatario'].replace(self.indiceTerminal))
        self.tInicio = np.array(
            (pedido['Fecha Pedido'] - self.fechaMin).dt.days)
        self.aranceles = np.array(pedido['Impuestos y aranceles'])

        # Indices de rutas disponibles

        self.numRuta = ruta[['Terminal Origen', 'Terminal Destino']
            ].drop_duplicates().shape[0]
        rutas = ruta[['Terminal Origen', 'Terminal Destino']
            ].drop_duplicates().replace(self.indiceTerminal)
        self.rutasDisp = list(
            zip(rutas['Terminal Origen'], rutas['Terminal Destino']))

        var_1_ubicacion = product(self.rutasDisp, range(
            self.auxTiempo), range(self.mercancia))
        var_1_ubicacion = [(i[0][0], i[0][1], i[1], i[2]) for i in var_1_ubicacion]
        self.var_1_ubicacion = tuple(zip(*var_1_ubicacion))

        var_2_ubicacion = product(self.rutasDisp, range(self.auxTiempo))
        var_2_ubicacion = [(i[0][0], i[0][1], i[1]) for i in var_2_ubicacion]
        self.var_2_ubicacion = tuple(zip(*var_2_ubicacion))

        self.var_3_ubicacion = self.var_2_ubicacion
    def crear_red(self, ruta):
        '''CREACION DE LA RED DE TRANSPORTE EXISTENTE'''
        
        nudos = self.terminalIndice.values()
        redTransporte = nx.MultiDiGraph()
        redTransporte.add_nodes_from(nudos)
        
        pos = nx.circular_layout(redTransporte, scale = 1)
        
        plt.figure(1, figsize = (12,12))
        
        dic_colores = {1: 'blue', 2: 'green', 3: 'red'}
        copia_ruta = ruta
        copia_ruta.replace({'Maritimo': 1, 'Ferrocarril': 2, 'Carretera': 3}, inplace = True)
        lista_colores = []
        
        for i in range(len(self.rutasDisp)):
            values = ruta.iloc[i].values
            copy_route_1 = copia_ruta.iloc[i].values
            
            lista_colores.append(dic_colores[copy_route_1[8]])
            redTransporte.add_edge(values[1], values[2], color = lista_colores[i])

        color_for_edges = nx.get_edge_attributes(redTransporte, 'color').values()
            
        nx.draw_networkx_nodes(redTransporte, pos, node_color = 'gold', node_size = 200)
        nx.draw_networkx_edges(redTransporte, pos, connectionstyle='arc3, rad = 0.05', width = 0.5, edge_color = color_for_edges, arrows=True)        
        nx.draw_networkx_labels(redTransporte, pos, font_size = 12)
        plt.title("Red de Transporte Intermodal")
        handles = [Line2D([], [], color = color, label = label) for color, label in zip(['blue', 'green', 'red'],['Maritimo','Ferrocarril','Carretera'])]
        plt.legend(handles=handles)
        plt.margins(x = 0.15, tight = True) 
        plt.show()       
       

    def constr_modelo(self):

        self.modelo_CVXPY()   

    def modelo_CVXPY(self):
        '''DEFINICION DEL MODELO MATEMATICO CON LAS FUNCIONES OBJETIVO Y RESTRICCIONES A RESOLVER CON CVXPY'''

        # Matriz binaria de 4 dimensiones con variables de decision

        self.var_1=cp.Variable(self.numRuta * self.auxTiempo * \
                             self.mercancia, boolean=True, name='x')
        self.x=np.zeros((self.auxTerminal, self.auxTerminal,
                        self.auxTiempo, self.mercancia)).astype('object')
        self.x[self.var_1_ubicacion]=list(self.var_1)

        # Matriz tridimensional con numero de contenedores

        self.var_2=cp.Variable(
            self.numRuta * self.auxTiempo, integer=True, name='y')
        self.y=np.zeros((self.auxTerminal, self.auxTerminal,
                        self.auxTiempo)).astype('object')
        self.y[self.var_2_ubicacion]=list(self.var_2)

        # Matriz tridimensional con ocupacion de rutas

        self.var_3=cp.Variable(
            self.numRuta * self.auxTiempo, boolean=True, name='z')
        self.z=np.zeros((self.auxTerminal, self.auxTerminal,
                        self.auxTiempo)).astype('object')
        self.z[self.var_3_ubicacion]=list(self.var_3)

        # Costes asociados a los depositos

        costeAlmacen, horaLlegada, tiempoEstancia=self.costes_almacen(self.x)

        # Funciones objetivo

        costeTransporte=np.sum(self.y * self.cTransporte) + \
                             np.sum(self.z * self.cFijosTransp)
        costeTransito=np.sum(
            np.sum(np.dot(self.x, self.cPedido), axis=2) * self.cTransito)
        cImpuestos=np.sum(self.aranceles * self.cPedido) + costeTransito
        ObjFun=cp.Minimize(costeTransporte + costeAlmacen + cImpuestos)
         # Restricciones

        restricciones=[]

        # 1. La mercancia debe ser transportada desde el nodo de origen hasta el destino

        restricciones += [np.sum(self.x[self.terminalOrigen[k], :, :, k])
                               == 1 for k in range(self.mercancia)]
        restricciones += [np.sum(self.x[:, self.terminalDestino[k], :, k])
                               == 1 for k in range(self.mercancia)]

        # 2. La mercancia no se puede entregar al origen ni enviar desde el destino

        restricciones += [np.sum(self.x[:, self.terminalOrigen[k], :, k])
                               == 0 for k in range(self.mercancia)]
        restricciones += [np.sum(self.x[self.terminalDestino[k], :, :, k])
                               == 0 for k in range(self.mercancia)]

        # 3. Restricciones de transbordo

        for k in range(self.mercancia):
            for j in range(self.auxTerminal):
                if (j != self.terminalOrigen[k]) & (j != self.terminalDestino[k]):
                    restricciones.append(
                        np.sum(self.x[:, j, :, k]) == np.sum(self.x[j, :, :, k]))

        # 4. La mercancia solo puede pasar una vez por un nodo

        restricciones += [np.sum(self.x[i, :, :, k]) <= 1 for k in range(self.mercancia)
                               for i in range(self.auxTerminal)]
        restricciones += [np.sum(self.x[:, j, :, k]) <= 1 for k in range(self.mercancia)
                               for j in range(self.auxTerminal)]

        # 5. Flujo de mercancia

        restricciones += [tiempoEstancia[j, k] >=
            0 for j in range(self.auxTerminal) for k in range(self.mercancia)]

        # 6. Restriccion para el numero de contenedores

        nContenedores=np.dot(self.x, self.vPedido) / self.vContenedor
        restricciones += [self.y[i, j, t] - nContenedores[i, j, t] >= 0 \
                        for i in range(self.auxTerminal) for j in range(self.auxTerminal) for t in
                        range(self.auxTiempo) if not isinstance(self.y[i, j, t] - nContenedores[i, j, t] >= 0, bool)]

        # 7. Verificacion de ruta en uso

        restricciones += [self.z[i, j, t] >= (np.sum(self.x[i, j, t, :]) * 10e-5) \
                        for i in range(self.auxTerminal) for j in range(self.auxTerminal) for t in
                        range(self.auxTiempo) if
                        not isinstance(self.z[i, j, t] >= (np.sum(self.x[i, j, t, :]) * 10e-5), bool)]

        # 8. Ventanas de tiempo

        restricciones += [np.sum(horaLlegada[:, self.terminalDestino[k], :, k]) <= self.fechaLimEntrega[k] for k in range(self.mercancia)
                        if not isinstance(np.sum(horaLlegada[:, self.terminalDestino[k], :, k]) <= self.fechaLimEntrega[k], bool)]
        modelo=cp.Problem(ObjFun, restricciones)


        self.ObjFun=ObjFun
        self.restricciones=restricciones
        self.modelo=modelo
        
    def resolver_modelo(self, solver=cp.CPLEX):
        try:
                self.valorObjFun=self.modelo.solve(solver, verbose=True)
                self.xs=np.zeros(
                    (self.auxTerminal, self.auxTerminal, self.auxTiempo, self.mercancia))
                self.xs[self.var_1_ubicacion]=self.var_1.value
                self.ys=np.zeros(
                    (self.auxTerminal, self.auxTerminal, self.auxTiempo))
                self.ys[self.var_2_ubicacion]=self.var_2.value
                self.zs=np.zeros(
                    (self.auxTerminal, self.auxTerminal, self.auxTiempo))
                self.zs[self.var_3_ubicacion]=self.var_3.value

        except:
            raise Exception(
                'No se puede resolver el modelo con este solver, seleccione otro')


        nonzeroX=list(zip(*np.nonzero(self.xs)))
        nonzeroX=sorted(nonzeroX, key=lambda x: x[2])
        nonzeroX=sorted(nonzeroX, key=lambda x: x[3])
        nonzeroX=list(map(lambda x: (self.terminalIndice[x[0]], self.terminalIndice[x[1]], \
                                       (self.fechaMin + pd.to_timedelta(x[2], unit='days')).date().isoformat(), x[3]), nonzeroX))

        self.cFinalAlmacen, horaLlegada, _=self.costes_almacen(self.xs)
        self.costeTransporte=np.sum(
            self.ys * self.cTransporte) + np.sum(self.zs * self.cFijosTransp)
        self.cImpuestos=np.sum(self.aranceles * self.cPedido) + np.sum(
            np.sum(np.dot(self.xs, self.cPedido), axis=2) * self.cTransito)

        self.solucion_={}
        self.horaLlegada_={}

        for i in range(self.mercancia):
            self.solucion_[
                'mercancia-' + str(i + 1)]=list(filter(lambda x: x[3] == i, nonzeroX))
            self.horaLlegada_['mercancia-' + str(i + 1)]=(self.fechaMin + pd.to_timedelta(
                np.sum(horaLlegada[:, self.terminalDestino[i], :, i]), unit='days')).date().isoformat()

    def salida_modelo(self):

        return self.valorObjFun, self.solucion_, self.horaLlegada_
        

    def costes_almacen(self, x):
        tiempoInicio=np.arange(self.auxTiempo).reshape(
            1, 1, self.auxTiempo, 1) * x
        matrFechaLlegada=tiempoInicio + self.tTransporte.reshape(
                self.auxTerminal, self.auxTerminal, self.auxTiempo, 1) * x
        horaLlegada=matrFechaLlegada.copy()
        matrFechaLlegada[:, self.terminalDestino.tolist(), :, range(self.mercancia)]=0
        tiempoEstancia=np.sum(tiempoInicio, axis=(1, 2)) - \
                        np.sum(matrFechaLlegada, axis=(0, 2))
        tiempoEstancia[self.terminalOrigen.tolist(), range(self.mercancia)
                                        ] -= self.tInicio
        costeAlmacen=np.sum(
            np.sum(tiempoEstancia * self.vPedido, axis=1) * self.cAlmacen)

        return costeAlmacen, horaLlegada, tiempoEstancia
    def solucion_txt(self, ruta, pedido):
        modoTransp=dict(
            zip(zip(ruta['Terminal Origen'], ruta['Terminal Destino']), ruta['Modo de Transporte']))
        txt="Solucion del modelo sincromodal simplificado de gestion del transporte intermodal"
        txt += "\nNumero de Pedidos: " + str(pedido['Numero de pedido'].count())
        txt += "\nCoste Global: " + \
            str(self.costeTransporte + self.cFinalAlmacen + self.cImpuestos)
        txt += "\nCoste del Transporte: " + str(self.costeTransporte)
        txt += "\nCostes de Almacen: " + str(self.cFinalAlmacen)
        txt += "\nImpuestos y Aranceles: " + str(self.cImpuestos)

        for i in range(pedido.shape[0]):
            txt += "\n------------------------------------"
            txt += "\nPedido " + str(i + 1) + \
                                    "  Mercancia: " + pedido['Mercancia'][i]
            txt += "\nFecha de Pedido: " + pd.to_datetime(pedido['Fecha Pedido']) \
                .iloc[i].date().isoformat()
            txt += "\nFecha de Entrega: " + \
                str(self.horaLlegada_['mercancia-' + str(i + 1)])
            txt += "\nRuta:"
            solucion=self.solucion_['mercancia-' + str(i + 1)]
            rutas_txt=''
            a=1

            for j in solucion:
                rutas_txt += "\n(" + str(a) + ")Fecha: " + j[2]
                rutas_txt += "  Origen: " + j[0]
                rutas_txt += "  Destino: " + j[1]
                rutas_txt += "  Modo: " + modoTransp[(j[0], j[1])]
                a += 1
            txt += rutas_txt

        return txt
    
    def datos_excel(filePath):
        pedido=pd.read_excel(filePath, sheet_name='Informacion Pedidos')
        ruta=pd.read_excel(filePath, sheet_name='Informacion Rutas')
        pedido['Impuestos y aranceles'][pedido['Tipo de Envio'] == 'Nacional'] = 0
        ruta['Cost']=ruta[ruta.columns[7:12]].sum(axis=1)
        ruta['Time']=np.ceil(ruta[ruta.columns[14:18]].sum(axis=1) / 24)
        ruta=ruta[list(ruta.columns[0:4]) + ['Costes Fijos Transporte', 'Time', 'Cost', 'Costes de Almacenamiento', 'Modo de Transporte', 'Tasas Internacionales']
                          + list(ruta.columns[-9:-2])]
        ruta=pd.melt(ruta, id_vars=ruta.columns[0:10], value_vars=ruta.columns[-7:],
                      var_name='Weekday', value_name='Feasibility')
        ruta['Weekday']=ruta['Weekday'].replace({'Lunes': 1, 'Martes': 2, 'Miercoles': 3, 'Jueves': 4, 'Viernes': 5, 'Sabado': 6, 'Domingo': 7})

        return pedido, ruta

        if __name__ == '__main__':
            pedido, ruta=datos_excel("Datos modelo.xlsx")
            m = MSS()
            #m.definir_param(self, ruta, pedido)
            m.definir_param(ruta, pedido)
            m.constr_modelo()
            m.resolver_modelo()
            txt = m.solucion_txt(self, ruta, pedido)
            txt = m.solucion_txt(ruta, pedido)
            m.crear_red(ruta)     
            text_file = open("C:/Users/Usuario/Documents/Enrutamiento/Seguros/Resultado.txt","w") 
            #text_file = open("Resultado.txt", "w+") # Genera un fichero de texto #(Resultado.txt), lo abre y comienza a escribir (w)
            text_file.write(txt) # Escribe el texto guardado en txt en el fichero
            text_file.close() # Cierra el fichero txt 
            # with open("Resultado.txt", "w") as text_file:
                #text_file.write(txt)
                #webbrowser.open("Resultado.txt") 
         

SyntaxError: invalid syntax (3820056545.py, line 433)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1296223089.py, line 1)